In [1]:
import os
from dotenv import load_dotenv
import pinecone
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

c:\Users\Aneta\OneDrive\Dokumenty\gsk_genAI_project\genAIproject\gsk_genAI_project\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import langchain
print(langchain.__version__)


1.0.3


In [3]:
SYSTEM_PROMPT = """
You are a research assistant helping the user understand scientific papers retrieved from arXiv.
Use the context retrieved from Pinecone to provide accurate, concise answers with citations.
If unsure, say so.
"""


In [10]:
# !capture --no-stderr
!pip install --quiet -U langchain_openai langchain_core langchain langchain-community langchain_pinecone beautifulsoup4 requests pinecone tabulate


In [ ]:
load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME", "arxiv")



# # --- Initialize Pinecone + models ---
# pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
# embeddings = AzureOpenAIEmbeddings(
#     deployment="text-embedding-3-small",
#     openai_api_key=AZURE_OPENAI_API_KEY,
#     openai_api_base=AZURE_OPENAI_ENDPOINT
# )

# if PINECONE_INDEX_NAME not in pinecone.list_indexes():
#     dim = len(embeddings.embed_query("test"))
#     pinecone.create_index(PINECONE_INDEX_NAME, dimension=dim)

# index = pinecone.Index(PINECONE_INDEX_NAME)
# store = Pinecone(index, embedding_function=embeddings.embed_query)

# print("✅ Connected to Pinecone index:", PINECONE_INDEX_NAME)

In [27]:
# pip install pinecone
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("arxiv")

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    deployment="text-embedding-3-small",
    model="text-embedding-3-small",
    openai_api_key=AZURE_OPENAI_API_KEY,
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_version="2025-01-01-preview"
)
